In [10]:
import os, json
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime

In [2]:
# Read API Key
config_file = {}
with open('config.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        key, value = line.strip().split('=')
        config_file[key] = value

API_KEY = config_file.get('API_KEY')

if API_KEY is None:
    raise ValueError('API_KEY not found')

In [3]:
def get_courses(key):
    request = "https://canvas.case.edu/api/v1/courses"
    headers = {"Authorization": f"Bearer {API_KEY}"}
    data = {
        "enrollment_type":"student",
        "enrollment_state":"active",
    }
    response = requests.get(request, params=data, headers=headers)
    json_object = response.json()
    return json.dumps(json_object)

def get_assignments(course_id, key):
    request = f"https://canvas.case.edu/api/v1/courses/{course_id}/assignments"
    headers = {"Authorization": f"Bearer {API_KEY}"}
    response = requests.get(request, headers=headers)
    json_object = response.json()
    return json.dumps(json_object)

def get_file(request,key):
    headers = {"Authorization": f"Bearer {API_KEY}"}
    response = requests.get(request, headers=headers)
    json_object = response.json()
    
    # Ensure the 'pdfs' folder exists
    if not os.path.exists('files'):
        os.makedirs('files')

    # Download PDF
    url = json_object.get('url')
    response = requests.get(url)
    file_path = os.path.join('files', json_object.get('filename'))

    # Save to 'pdfs' folder
    with open(file_path, 'wb') as f:
        f.write(response.content)

    # Return dictionary entry
    result = {
        'displayname': json_object.get('display_name'),
        'filename': file_path
    }
    
    return result




In [4]:
def parse_assignment_text(assignment_description):
    soup = BeautifulSoup(assignment_description, 'html.parser')

    # Extracting all text
    text = soup.stripped_strings
    plaintext = " ".join(text)

    # Extracting API endpoints
    api_endpoints = [tag['data-api-endpoint'] for tag in soup.find_all('a') if tag.has_attr('data-api-endpoint')]

    return plaintext, api_endpoints



In [20]:
courses = get_courses(API_KEY)
courses = json.loads(courses)
processed_assignments = []
for course in courses:
    course_id = course.get('id', 'N/A')
    course_name = course.get('name', 'N/A')
    #print(f"Course ID: {course_id}, Course Name: {course_name}")
    assignments = get_assignments(course.get('id', 'N/A'),API_KEY)
    assignments = json.loads(assignments)
    for assignment in assignments:
        assignment_name = assignment.get('name','N/A')
        assignment_description = assignment.get('description',None)
        if assignment.get('is_quiz_assignment'):
            continue
        assignment_description, endpoints = parse_assignment_text(assignment_description)
        endpoints = [] if endpoints is None else endpoints
        
        due_date = assignment.get('due_at', None)
        if due_date is not None:
            due_date = datetime.strptime(due_date, '%Y-%m-%dT%H:%M:%SZ')
        #print(f"{assignment_name}\n{assignment_description} (num API endpoints: {endpoints})")
        #all_endpoints.extend(endpoints)
        processed_assignments.append({
            'course': course_name,
            'assignment_name' : assignment_name,
            'due_date' : due_date,
            'content' : assignment_description,
            'files' : [get_file(endpoint,API_KEY) for endpoint in endpoints]
        })
print(processed_assignments)

[{'course': 'Bayesian Scientific Computing (100/10794)', 'assignment_name': 'Homework 1', 'due_date': datetime.datetime(2023, 10, 3, 3, 59, 59), 'content': 'HW1.pdf', 'files': [{'displayname': 'HW1.pdf', 'filename': 'files/HW1.pdf'}]}, {'course': 'Bayesian Scientific Computing (100/10794)', 'assignment_name': 'Homework 2', 'due_date': datetime.datetime(2023, 10, 18, 3, 59, 59), 'content': 'HW2.pdf HW2.tex', 'files': [{'displayname': 'HW2.pdf', 'filename': 'files/HW2.pdf'}, {'displayname': 'HW2.tex', 'filename': 'files/HW2.tex'}]}, {'course': 'Fundamentals of Analysis I (100/1187)', 'assignment_name': 'Homework 1', 'due_date': datetime.datetime(2023, 9, 14, 3, 59, 59), 'content': 'Hello Everyone, HW 1 has been uploaded and will be due on Sep 13, 2023 by 11:59 p.m. No late assignments will be accepted. HW 1 Math 321.pdf', 'files': [{'displayname': 'HW 1 Math 321.pdf', 'filename': 'files/HW+1+Math+321.pdf'}]}, {'course': 'Fundamentals of Analysis I (100/1187)', 'assignment_name': 'Homewor

In [22]:
current_assignments = [assignment for assignment in processed_assignments if assignment['due_date'] is not None and assignment['due_date'] > datetime.now()]


In [24]:
assgns = [f"{assignment['course']}, {assignment['assignment_name']}\n{assignment['due_date']}" for assignment in current_assignments]
for a in assgns:
    print(a)

Bayesian Scientific Computing (100/10794), Homework 2
2023-10-18 03:59:59
Fundamentals of Analysis I (100/1187), Homework 4
2023-10-17 03:59:59
Heat Transfer (100/4773), Assignment 5
2023-10-12 03:59:59
Heat Transfer (100/4773), Assignment 6
2023-10-21 03:59:59
